In [11]:
from google.oauth2 import service_account
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.auth import exceptions
from google.analytics.data_v1beta.types import DateRange, Metric, Dimension, RunReportRequest

# Ruta al archivo JSON con las credenciales de la cuenta de servicio
KEY_FILE_LOCATION = 'C:\\Users\\ehmet\\OneDrive\\Escritorio\\Proyectos de data\\data_analytics\\data-analytics-project-409620-abc0407700dd.json'  # Actualiza la ruta

# ID de la propiedad de Google Analytics 4
PROPERTY_ID = '421423594'  # Reemplaza con tu identificador de propiedad

# Configurar las credenciales
try:
    credentials = service_account.Credentials.from_service_account_file(
        KEY_FILE_LOCATION,
        scopes=['https://www.googleapis.com/auth/analytics.readonly']
    )
except exceptions.GoogleAuthError as auth_error:
    print(f"Error de autenticación: {auth_error}")
    raise

# Crear un cliente de Google Analytics Data API
client = BetaAnalyticsDataClient(credentials=credentials)

try:
    # Realizar una consulta de ejemplo
    response = client.run_report(
        request=RunReportRequest(
            property=f'properties/{PROPERTY_ID}',
            date_ranges=[DateRange(start_date='2023-01-01', end_date='2023-01-07')],
            dimensions=[Dimension(name='date')],
            metrics=[Metric(name='engagedSessions')],
        )
    )

    # Imprimir la respuesta
    print(response)

except Exception as e:
    # Imprimir información de error
    print(f"Error: {e}")


dimension_headers {
  name: "date"
}
metric_headers {
  name: "engagedSessions"
  type_: TYPE_INTEGER
}
metadata {
  currency_code: "USD"
  time_zone: "America/Buenos_Aires"
}
kind: "analyticsData#runReport"

